# Evolver Loop 4 Analysis

## Problem: Submission Failed with Overlapping Trees in Group 042

The better baseline (70.647306) from snapshot 21165874980 was rejected by Kaggle due to overlapping trees in group 042 (N=42). Our local validation didn't catch this.

**Key Questions:**
1. Why did our local validation miss the overlap?
2. What is the actual overlap in group 042?
3. Can we fix it?
4. What is our best VALID submission?

In [1]:
import pandas as pd
import numpy as np
from shapely.geometry import Polygon
from shapely import affinity
import os

# Tree geometry
TX = [0, 0.125, 0.0625, 0.2, 0.1, 0.35, 0.075, 0.075, -0.075, -0.075, -0.35, -0.1, -0.2, -0.0625, -0.125]
TY = [0.8, 0.5, 0.5, 0.25, 0.25, 0, 0, -0.2, -0.2, 0, 0, 0.25, 0.25, 0.5, 0.5]
BASE_TREE = Polygon(zip(TX, TY))

def parse_value(s):
    if isinstance(s, str) and s.startswith('s'):
        return float(s[1:])
    return float(s)

def create_tree_polygon(x, y, deg):
    tree = affinity.rotate(BASE_TREE, deg, origin=(0, 0))
    tree = affinity.translate(tree, x, y)
    return tree

print("Functions defined")

Functions defined


In [2]:
# Load the failed submission
failed_file = '/home/nonroot/snapshots/santa-2025/21165874980/code/submission_candidates/candidate_001.csv'
df = pd.read_csv(failed_file)
print(f"Loaded {len(df)} rows")

Loaded 20100 rows


In [3]:
# Check group 042 specifically with STRICTER overlap detection
prefix = '042_'
n_trees = df[df['id'].str.startswith(prefix)]
print(f"N=42 has {len(n_trees)} trees")

polygons = []
for _, row in n_trees.iterrows():
    x = parse_value(row['x'])
    y = parse_value(row['y'])
    deg = parse_value(row['deg'])
    poly = create_tree_polygon(x, y, deg)
    polygons.append((row['id'], poly))

# Check all pairs with VERY strict threshold
overlaps = []
for i in range(len(polygons)):
    for j in range(i+1, len(polygons)):
        id_i, poly_i = polygons[i]
        id_j, poly_j = polygons[j]
        if poly_i.intersects(poly_j):
            inter = poly_i.intersection(poly_j)
            if inter.area > 0:  # ANY overlap
                overlaps.append((id_i, id_j, inter.area))
                
print(f"Found {len(overlaps)} overlaps in N=42:")
for id_i, id_j, area in overlaps:
    print(f"  {id_i} and {id_j}: area = {area:.15f}")

N=42 has 42 trees
Found 0 overlaps in N=42:


In [4]:
# Check with even stricter - use buffer to detect near-overlaps
print("\nChecking with buffer for near-overlaps:")
for i in range(len(polygons)):
    for j in range(i+1, len(polygons)):
        id_i, poly_i = polygons[i]
        id_j, poly_j = polygons[j]
        # Check if they're touching or very close
        dist = poly_i.distance(poly_j)
        if dist < 1e-6:
            inter = poly_i.intersection(poly_j)
            print(f"  {id_i} and {id_j}: distance = {dist:.15f}, intersection area = {inter.area:.15f}")


Checking with buffer for near-overlaps:
  042_0 and 042_1: distance = 0.000000000000015, intersection area = 0.000000000000000
  042_0 and 042_7: distance = 0.000000000000019, intersection area = 0.000000000000000
  042_0 and 042_30: distance = 0.000000000066624, intersection area = 0.000000000000000
  042_0 and 042_35: distance = 0.000000000000037, intersection area = 0.000000000000000
  042_1 and 042_39: distance = 0.000000000000043, intersection area = 0.000000000000000
  042_2 and 042_6: distance = 0.000000000000079, intersection area = 0.000000000000000
  042_2 and 042_17: distance = 0.000000000000002, intersection area = 0.000000000000000
  042_2 and 042_36: distance = 0.000000000000009, intersection area = 0.000000000000000
  042_3 and 042_10: distance = 0.000000000000002, intersection area = 0.000000000000000
  042_3 and 042_12: distance = 0.000000000000002, intersection area = 0.000000000000000
  042_3 and 042_17: distance = 0.000000000000000, intersection area = 0.0000000000

In [5]:
# Let's check ALL groups for overlaps with zero tolerance
def check_all_overlaps_strict(df):
    overlaps_found = []
    for n in range(1, 201):
        prefix = f"{n:03d}_"
        n_trees = df[df['id'].str.startswith(prefix)]
        if len(n_trees) != n:
            continue
        polygons = []
        for _, row in n_trees.iterrows():
            x = parse_value(row['x'])
            y = parse_value(row['y'])
            deg = parse_value(row['deg'])
            poly = create_tree_polygon(x, y, deg)
            polygons.append((row['id'], poly))
        for i in range(len(polygons)):
            for j in range(i+1, len(polygons)):
                id_i, poly_i = polygons[i]
                id_j, poly_j = polygons[j]
                if poly_i.intersects(poly_j) and not poly_i.touches(poly_j):
                    inter = poly_i.intersection(poly_j)
                    if inter.area > 0:  # ANY positive area
                        overlaps_found.append((n, id_i, id_j, inter.area))
    return overlaps_found

print("Checking ALL groups for overlaps (strict)...")
all_overlaps = check_all_overlaps_strict(df)
print(f"Found {len(all_overlaps)} total overlaps:")
for n, id_i, id_j, area in all_overlaps:
    print(f"  N={n}: {id_i} and {id_j}, area = {area:.15f}")

Checking ALL groups for overlaps (strict)...


Found 0 total overlaps:


In [6]:
# Now let's check our KNOWN GOOD submissions
print("\n" + "="*60)
print("Checking known good submissions:")
print("="*60)

# The 70.658891 submission that passed LB
good_file = '/home/nonroot/snapshots/santa-2025/21164519357/code/exploration/santa-2025.csv'
df_good = pd.read_csv(good_file)
print(f"\nChecking {good_file}...")
good_overlaps = check_all_overlaps_strict(df_good)
print(f"Found {len(good_overlaps)} overlaps")

# The original baseline
baseline_file = '/home/nonroot/snapshots/santa-2025/21116303805/code/preoptimized/santa-2025.csv'
df_baseline = pd.read_csv(baseline_file)
print(f"\nChecking {baseline_file}...")
baseline_overlaps = check_all_overlaps_strict(df_baseline)
print(f"Found {len(baseline_overlaps)} overlaps")


Checking known good submissions:

Checking /home/nonroot/snapshots/santa-2025/21164519357/code/exploration/santa-2025.csv...


Found 0 overlaps

Checking /home/nonroot/snapshots/santa-2025/21116303805/code/preoptimized/santa-2025.csv...


Found 0 overlaps


In [7]:
# Calculate scores for the good submissions
def calculate_score(df):
    total_score = 0
    for n in range(1, 201):
        prefix = f"{n:03d}_"
        n_trees = df[df['id'].str.startswith(prefix)]
        if len(n_trees) != n:
            continue
        all_coords = []
        for _, row in n_trees.iterrows():
            x = parse_value(row['x'])
            y = parse_value(row['y'])
            deg = parse_value(row['deg'])
            poly = create_tree_polygon(x, y, deg)
            coords = np.array(poly.exterior.coords)
            all_coords.append(coords)
        all_coords = np.vstack(all_coords)
        x_range = all_coords[:, 0].max() - all_coords[:, 0].min()
        y_range = all_coords[:, 1].max() - all_coords[:, 1].min()
        side = max(x_range, y_range)
        total_score += side**2 / n
    return total_score

print("\nScores:")
print(f"Good submission (70.658891): {calculate_score(df_good):.6f}")
print(f"Baseline (70.676102): {calculate_score(df_baseline):.6f}")
print(f"Failed submission: {calculate_score(df):.6f}")


Scores:


Good submission (70.658891): 70.658891


Baseline (70.676102): 70.676102


Failed submission: 70.647306


In [8]:
# Let's look at what the Kaggle validator might be doing differently
# Perhaps they use a different precision or algorithm

# Check the exact coordinates in group 042
print("\nExact coordinates for N=42:")
for _, row in n_trees.head(10).iterrows():
    x = parse_value(row['x'])
    y = parse_value(row['y'])
    deg = parse_value(row['deg'])
    print(f"{row['id']}: x={x:.15f}, y={y:.15f}, deg={deg:.15f}")


Exact coordinates for N=42:
042_0: x=1.747533353797365, y=-0.611676768965902, deg=84.643132034986920
042_1: x=1.747801675800098, y=-0.007535041242537, deg=126.027371818347092
042_2: x=-1.142327623636033, y=0.748057350043896, deg=66.743364515377237
042_3: x=0.040329541205713, y=0.698664845175687, deg=96.190503401038725
042_4: x=-0.836786630187964, y=1.351307626548424, deg=16623.662758960035717
042_5: x=-0.924439691494856, y=-0.789981953403887, deg=249.288487955135196
042_6: x=-1.650886555946404, y=0.410841532391147, deg=23.629377894724080
042_7: x=0.804120931319439, y=-1.039937420311431, deg=287.859232904795533
042_8: x=-0.538962792887715, y=-1.896394610900443, deg=246.366486619751640
042_9: x=-0.340219700241197, y=-1.446684061242404, deg=65.641764500625158


In [9]:
# Let's try using a different overlap detection method
# Kaggle might use a simpler/different algorithm

from shapely.validation import make_valid

def check_overlaps_alternative(df, n):
    """Check overlaps using different methods"""
    prefix = f"{n:03d}_"
    n_trees = df[df['id'].str.startswith(prefix)]
    
    polygons = []
    for _, row in n_trees.iterrows():
        x = parse_value(row['x'])
        y = parse_value(row['y'])
        deg = parse_value(row['deg'])
        poly = create_tree_polygon(x, y, deg)
        polygons.append((row['id'], poly))
    
    print(f"\nN={n}: Checking {len(polygons)} trees")
    
    # Method 1: Standard intersection
    for i in range(len(polygons)):
        for j in range(i+1, len(polygons)):
            id_i, poly_i = polygons[i]
            id_j, poly_j = polygons[j]
            
            # Check intersection
            if poly_i.intersects(poly_j):
                inter = poly_i.intersection(poly_j)
                if inter.area > 1e-15:  # Very strict
                    print(f"  OVERLAP: {id_i} & {id_j}, area={inter.area:.20f}")
                elif not poly_i.touches(poly_j):
                    print(f"  NEAR: {id_i} & {id_j}, touches={poly_i.touches(poly_j)}, area={inter.area:.20f}")

check_overlaps_alternative(df, 42)


N=42: Checking 42 trees


In [10]:
# Let's check if there's a precision issue with the coordinates
# Maybe the 's' prefix parsing is causing issues

print("\nRaw data for N=42:")
for _, row in n_trees.head(5).iterrows():
    print(f"{row['id']}: x='{row['x']}', y='{row['y']}', deg='{row['deg']}'")

# Check if there are any unusual values
print("\nChecking for unusual values in N=42:")
for _, row in n_trees.iterrows():
    x = parse_value(row['x'])
    y = parse_value(row['y'])
    deg = parse_value(row['deg'])
    if abs(x) > 100 or abs(y) > 100:
        print(f"  Large coordinate: {row['id']}: x={x}, y={y}")
    if deg < 0 or deg > 360:
        print(f"  Unusual angle: {row['id']}: deg={deg}")


Raw data for N=42:
042_0: x='s1.7475333537973645', y='s-0.6116767689659015', deg='s84.64313203498692'
042_1: x='s1.7478016758000978', y='s-0.007535041242536562', deg='s126.02737181834709'
042_2: x='s-1.1423276236360327', y='s0.7480573500438955', deg='s66.74336451537724'
042_3: x='s0.04032954120571284', y='s0.6986648451756872', deg='s96.19050340103873'
042_4: x='s-0.8367866301879636', y='s1.3513076265484243', deg='s16623.662758960036'

Checking for unusual values in N=42:
  Unusual angle: 042_4: deg=16623.662758960036
  Unusual angle: 042_32: deg=-833.0016095256168


In [ ]:
# Let's use a completely different approach - check using the exact same method
# that the competition likely uses

# The competition scoring code probably uses a simple polygon intersection check
# Let's implement it from scratch

def point_in_polygon(point, polygon_coords):
    """Ray casting algorithm"""
    x, y = point
    n = len(polygon_coords)
    inside = False
    j = n - 1
    for i in range(n):
        xi, yi = polygon_coords[i]
        xj, yj = polygon_coords[j]
        if ((yi > y) != (yj > y)) and (x < (xj - xi) * (y - yi) / (yj - yi) + xi):
            inside = not inside
        j = i
    return inside

def get_tree_coords(x, y, deg):
    """Get rotated and translated tree coordinates"""
    import math
    rad = math.radians(deg)
    cos_a = math.cos(rad)
    sin_a = math.sin(rad)
    coords = []
    for tx, ty in zip(TX, TY):
        # Rotate
        rx = tx * cos_a - ty * sin_a
        ry = tx * sin_a + ty * cos_a
        # Translate
        coords.append((rx + x, ry + y))
    return coords

# Check if any vertex of one tree is inside another
def check_vertex_overlap(df, n):
    prefix = f"{n:03d}_"
    n_trees = df[df['id'].str.startswith(prefix)]
    
    trees = []
    for _, row in n_trees.iterrows():
        x = parse_value(row['x'])
        y = parse_value(row['y'])
        deg = parse_value(row['deg'])
        coords = get_tree_coords(x, y, deg)
        trees.append((row['id'], coords))
    
    print(f"\nN={n}: Vertex-in-polygon check")
    for i in range(len(trees)):
        for j in range(i+1, len(trees)):
            id_i, coords_i = trees[i]
            id_j, coords_j = trees[j]
            
            # Check if any vertex of i is inside j
            for pt in coords_i:
                if point_in_polygon(pt, coords_j):
                    print(f"  OVERLAP: vertex of {id_i} inside {id_j}")
                    break
            
            # Check if any vertex of j is inside i
            for pt in coords_j:
                if point_in_polygon(pt, coords_i):
                    print(f"  OVERLAP: vertex of {id_j} inside {id_i}")
                    break

check_vertex_overlap(df, 42)

In [11]:
# The unusual angles might be the issue!
# Let's check if normalizing angles to 0-360 changes anything

def normalize_angle(deg):
    """Normalize angle to 0-360 range"""
    return deg % 360

# Check all unusual angles in the failed submission
print("All unusual angles in failed submission:")
for _, row in df.iterrows():
    deg = parse_value(row['deg'])
    if deg < 0 or deg > 360:
        norm_deg = normalize_angle(deg)
        print(f"  {row['id']}: deg={deg:.6f} -> normalized={norm_deg:.6f}")

All unusual angles in failed submission:
  015_12: deg=9648.500796 -> normalized=288.500796
  018_17: deg=-4702.325187 -> normalized=337.674813
  020_8: deg=-3126.799199 -> normalized=113.200801
  021_9: deg=3524.351716 -> normalized=284.351716
  021_10: deg=-23521.963112 -> normalized=238.036888
  025_13: deg=7319.841287 -> normalized=119.841287
  029_5: deg=1597.714371 -> normalized=157.714371
  029_12: deg=4298.615864 -> normalized=338.615864
  029_15: deg=40385.353513 -> normalized=65.353513
  029_16: deg=-2226.370624 -> normalized=293.629376
  029_23: deg=3572.725333 -> normalized=332.725333
  029_25: deg=10972.411301 -> normalized=172.411301
  030_20: deg=2723.613214 -> normalized=203.613214
  034_1: deg=-2540.517872 -> normalized=339.482128
  034_27: deg=-1192.747921 -> normalized=247.252079
  036_19: deg=-8303.473700 -> normalized=336.526300
  036_34: deg=-11003.165564 -> normalized=156.834436
  037_4: deg=-5826.438441 -> normalized=293.561559
  039_17: deg=9608.373145 -> norma

  160_0: deg=-1991.850566 -> normalized=168.149434
  160_7: deg=2327.861552 -> normalized=167.861552
  160_12: deg=6828.578522 -> normalized=348.578522
  160_23: deg=-550.978795 -> normalized=169.021205
  160_39: deg=1248.680756 -> normalized=168.680756
  160_47: deg=1969.875422 -> normalized=169.875422
  160_52: deg=1788.708721 -> normalized=348.708721
  160_79: deg=-551.754204 -> normalized=168.245796
  160_83: deg=-22499.640443 -> normalized=180.359557
  160_88: deg=1968.189694 -> normalized=168.189694
  160_99: deg=2154.556525 -> normalized=354.556525
  160_103: deg=2148.078667 -> normalized=348.078667
  160_105: deg=6468.989200 -> normalized=348.989200
  160_111: deg=-2856.370622 -> normalized=23.629378
  160_122: deg=1078.616683 -> normalized=358.616683
  160_123: deg=1823.629378 -> normalized=23.629378
  160_140: deg=-1092.496813 -> normalized=347.503187
  160_151: deg=-1450.978795 -> normalized=349.021205
  161_132: deg=-4311.595533 -> normalized=8.404467
  161_154: deg=-1628.2

In [12]:
# Let's check if the good submission has any unusual angles
print("Unusual angles in GOOD submission (70.658891):")
count = 0
for _, row in df_good.iterrows():
    deg = parse_value(row['deg'])
    if deg < 0 or deg > 360:
        count += 1
        if count <= 20:
            print(f"  {row['id']}: deg={deg:.6f}")
print(f"Total unusual angles: {count}")

print("\nUnusual angles in BASELINE submission (70.676102):")
count = 0
for _, row in df_baseline.iterrows():
    deg = parse_value(row['deg'])
    if deg < 0 or deg > 360:
        count += 1
        if count <= 20:
            print(f"  {row['id']}: deg={deg:.6f}")
print(f"Total unusual angles: {count}")

Unusual angles in GOOD submission (70.658891):
  005_4: deg=12810.004583
  009_2: deg=4344.502311
  014_1: deg=-116.567214
  014_10: deg=-71.403231
  014_5: deg=-116.568906
  014_6: deg=-116.567214
  014_7: deg=-70.554362
  014_8: deg=-23.629378
  014_9: deg=-68.939683
  015_7: deg=9648.500796
  018_17: deg=-4709.593369
  020_16: deg=-3126.799199
  021_17: deg=3524.351716
  021_18: deg=-23521.963112
  025_20: deg=7319.841287
  029_13: deg=1597.714371
  029_2: deg=4298.615864
  029_22: deg=40385.353513
  029_23: deg=-2226.370624
  029_4: deg=3572.725333


Total unusual angles: 1091

Unusual angles in BASELINE submission (70.676102):
  005_4: deg=12809.955237
  009_2: deg=4344.563746
  014_1: deg=-116.559442
  014_2: deg=-116.583820
  014_3: deg=-70.549011
  014_4: deg=-23.629378
  014_5: deg=-69.186569
  014_6: deg=-71.583820
  014_13: deg=-116.565047
  015_7: deg=9649.847936
  018_17: deg=-4709.593369
  020_16: deg=-3126.272413
  021_17: deg=3525.111956
  021_18: deg=-23521.777225
  025_20: deg=7319.841287
  029_3: deg=40382.707024
  029_4: deg=-2226.370622
  029_10: deg=4298.629377
  029_12: deg=3572.626399
  029_14: deg=10972.367591


Total unusual angles: 1191


In [13]:
# Let's check if Kaggle uses a different tolerance for overlaps
# Maybe they consider "touching" as overlapping

# Check N=42 with different methods
prefix = '042_'
n_trees = df[df['id'].str.startswith(prefix)]

# Build polygons with high precision
from decimal import Decimal, getcontext
getcontext().prec = 50

polygons_42 = []
for _, row in n_trees.iterrows():
    x = parse_value(row['x'])
    y = parse_value(row['y'])
    deg = parse_value(row['deg'])
    poly = create_tree_polygon(x, y, deg)
    polygons_42.append((row['id'], poly))

# Check for ANY intersection (including touching)
print("Checking N=42 for ANY intersection:")
for i in range(len(polygons_42)):
    for j in range(i+1, len(polygons_42)):
        id_i, poly_i = polygons_42[i]
        id_j, poly_j = polygons_42[j]
        if poly_i.intersects(poly_j):
            inter = poly_i.intersection(poly_j)
            # Check if it's more than just touching
            if inter.geom_type != 'Point' and inter.geom_type != 'LineString':
                print(f"  {id_i} & {id_j}: {inter.geom_type}, area={inter.area:.20f}")

Checking N=42 for ANY intersection:


In [14]:
# Let's compare N=42 between the failed and good submissions
# Maybe the failed submission has a different configuration for N=42

prefix = '042_'
n_trees_failed = df[df['id'].str.startswith(prefix)]
n_trees_good = df_good[df_good['id'].str.startswith(prefix)]

print("N=42 comparison:")
print(f"Failed submission: {len(n_trees_failed)} trees")
print(f"Good submission: {len(n_trees_good)} trees")

# Calculate bounding box for each
def get_bbox_score(df_subset):
    all_coords = []
    for _, row in df_subset.iterrows():
        x = parse_value(row['x'])
        y = parse_value(row['y'])
        deg = parse_value(row['deg'])
        poly = create_tree_polygon(x, y, deg)
        coords = np.array(poly.exterior.coords)
        all_coords.append(coords)
    all_coords = np.vstack(all_coords)
    x_range = all_coords[:, 0].max() - all_coords[:, 0].min()
    y_range = all_coords[:, 1].max() - all_coords[:, 1].min()
    side = max(x_range, y_range)
    return side**2 / len(df_subset)

print(f"\\nFailed N=42 score: {get_bbox_score(n_trees_failed):.10f}")
print(f"Good N=42 score: {get_bbox_score(n_trees_good):.10f}")

N=42 comparison:
Failed submission: 42 trees
Good submission: 42 trees
\nFailed N=42 score: 0.3668386121
Good N=42 score: 0.3668386121


In [15]:
# Let's check if the issue is with how we're reading the 's' prefix
# Maybe Kaggle interprets it differently

# Check the raw format
print("Sample raw data from failed submission:")
print(df.head(3).to_string())

print("\\nSample raw data from good submission:")
print(df_good.head(3).to_string())

# Check if there's any difference in format
print("\\nColumn dtypes - failed:")
print(df.dtypes)
print("\\nColumn dtypes - good:")
print(df_good.dtypes)

Sample raw data from failed submission:
      id                     x                      y                  deg
0  001_0  s-48.196086194214246    s58.770984615214225                s45.0
1  002_0   s0.1540970696213643  s-0.03854074269478543  s203.62937773065684
2  002_1  s-0.1540970696213643   s-0.5614592573052146  s23.629377730656792
\nSample raw data from good submission:
      id                          x                         y                        deg
0  001_0  s-48.19608619421424577922  s58.77098461521422478882   s45.00000000000000000000
1  002_0    s0.15409706962135588659  s-0.03854074269479464826  s203.62937773065684154972
2  002_1   s-0.15409706962137284525  s-0.56145925730522405761   s23.62937773065679181173
\nColumn dtypes - failed:
id     object
x      object
y      object
deg    object
dtype: object
\nColumn dtypes - good:
id     object
x      object
y      object
deg    object
dtype: object


In [17]:
# The issue is clear now - the failed submission has lower precision
# Let's check the precision difference more carefully

print("Precision comparison for first few rows:")
for i in range(min(5, len(df))):
    row_f = df.iloc[i]
    row_g = df_good.iloc[i]
    print(f"\nRow {i} ({row_f['id']}):")
    print(f"  Failed x: {row_f['x']} ({len(str(row_f['x']))} chars)")
    print(f"  Good x:   {row_g['x']} ({len(str(row_g['x']))} chars)")

# The good submission has ~22 decimal places, failed has ~16
# This precision difference might cause Kaggle's validator to detect overlaps

Precision comparison for first few rows:

Row 0 (001_0):
  Failed x: s-48.196086194214246 (20 chars)
  Good x:   s-48.19608619421424577922 (25 chars)

Row 1 (002_0):
  Failed x: s0.1540970696213643 (19 chars)
  Good x:   s0.15409706962135588659 (23 chars)

Row 2 (002_1):
  Failed x: s-0.1540970696213643 (20 chars)
  Good x:   s-0.15409706962137284525 (24 chars)

Row 3 (003_0):
  Failed x: s1.123655816140301 (18 chars)
  Good x:   s1.12365581614030096702 (23 chars)

Row 4 (003_1):
  Failed x: s1.23405569584216 (17 chars)
  Good x:   s1.23405569584216001644 (23 chars)


In [18]:
# Let's create a hybrid submission:
# Use the good submission (70.658891) as base, but check if any N values
# from the failed submission (70.647306) are actually better AND valid

# First, let's see which N values are different
print("Comparing N values between submissions:")
differences = []
for n in range(1, 201):
    prefix = f"{n:03d}_"
    n_failed = df[df['id'].str.startswith(prefix)]
    n_good = df_good[df_good['id'].str.startswith(prefix)]
    
    score_failed = get_bbox_score(n_failed)
    score_good = get_bbox_score(n_good)
    
    if abs(score_failed - score_good) > 1e-10:
        differences.append((n, score_failed, score_good, score_good - score_failed))

print(f"Found {len(differences)} N values with different scores:")
for n, sf, sg, diff in sorted(differences, key=lambda x: x[3], reverse=True)[:20]:
    print(f"  N={n}: failed={sf:.10f}, good={sg:.10f}, diff={diff:+.10f}")

Comparing N values between submissions:


Found 102 N values with different scores:
  N=128: failed=0.3409736668, good=0.3437158332, diff=+0.0027421664
  N=94: failed=0.3500758984, good=0.3522710240, diff=+0.0021951256
  N=157: failed=0.3402039442, good=0.3413598431, diff=+0.0011558989
  N=14: failed=0.3695427692, good=0.3704540682, diff=+0.0009112990
  N=11: failed=0.3749237191, good=0.3757360704, diff=+0.0008123512
  N=193: failed=0.3333412670, good=0.3337596645, diff=+0.0004183975
  N=137: failed=0.3428104198, good=0.3431769093, diff=+0.0003664895
  N=123: failed=0.3475645691, good=0.3479042376, diff=+0.0003396684
  N=59: failed=0.3605834508, good=0.3608552211, diff=+0.0002717703
  N=115: failed=0.3433325056, good=0.3435696546, diff=+0.0002371490
  N=101: failed=0.3488013233, good=0.3490352807, diff=+0.0002339574
  N=192: failed=0.3350774194, good=0.3353007688, diff=+0.0002233494
  N=89: failed=0.3467170376, good=0.3468987685, diff=+0.0001817309
  N=32: failed=0.3654172496, good=0.3655921341, diff=+0.0001748845
  N=76: fail

In [ ]:
# STRATEGY: The good submission (70.658891) passed validation
# We should submit it to confirm LB score, then try to improve

# For now, let's just use the good submission as our baseline
# and focus on optimization strategies

# Copy the good submission to /home/submission/
import shutil
shutil.copy(good_file, '/home/submission/submission.csv')
print(f"Copied good submission to /home/submission/submission.csv")
print(f"Score: 70.658891")

# Verify it's correct
df_verify = pd.read_csv('/home/submission/submission.csv')
score_verify = calculate_score(df_verify)
print(f"Verified score: {score_verify:.6f}")

In [ ]:
# Let's also look for other pre-optimized submissions in the snapshots
# that might have better scores AND high precision

import glob

print("Searching for all CSV files in snapshots...")
all_csvs = glob.glob('/home/nonroot/snapshots/santa-2025/*/code/**/*.csv', recursive=True)
print(f"Found {len(all_csvs)} CSV files")

# Check each one for score and precision
valid_submissions = []
for csv_path in all_csvs:
    try:
        df_test = pd.read_csv(csv_path)
        if len(df_test) == 20100 and 'id' in df_test.columns:
            # Check precision of first row
            first_x = str(df_test.iloc[0]['x'])
            precision = len(first_x.split('.')[-1]) if '.' in first_x else 0
            
            # Calculate score
            score = calculate_score(df_test)
            
            # Check for overlaps
            overlaps = check_all_overlaps_strict(df_test)
            
            if len(overlaps) == 0:
                valid_submissions.append((csv_path, score, precision))
                if score < 70.66:
                    print(f"  {csv_path}: score={score:.6f}, precision={precision}")
    except Exception as e:
        pass

print(f"\nFound {len(valid_submissions)} valid submissions")